In [ ]:
import numpy as np
import math
from scipy.fftpack import fft

Sample data (canonical m=2 example from SKPrime):

In [ ]:
dv = np.array([-0.2517+0.3129j, 0.2307-0.4667j])
qv = np.array([0.2377, 0.1557])
m = dv.size

Some needed functions.

In [ ]:
def polyval(c, z):
    "Polynomial evaluation."
    
    p = np.zeros(z.shape, dtype=z.dtype)
    for a in c:
        p = p*z + a
        
    return p

In [ ]:
i2pi = 2.0j*math.pi

def polesInHoles(z):
    "Put a pole in each circle center."
    
    w = np.zeros(z.shape, z.dtype)
    for d, q in zip(dv, qv):
        w += q/(z - d)/i2pi
        
    return w

# real part
g = lambda z: polesInHoles(z).real

Sample $N$ points on each boundary.

In [ ]:
M = 129

circ = np.exp(i2pi*np.arange(M, dtype=np.double)/M)
zb = np.empty((M, m+1), dtype=np.complex_)
zb[:,0] = circ
for d, q, j in zip(dv, qv, range(m)):
    zb[:,j+1] = d + q*circ

Create interpolation coefficients via FFT.

In [ ]:
a = np.empty(zb.shape, dtype=np.complex_)
for j in range(m+1):
    a[:,j] = fft(g(zb[:,j]))/M

In [ ]:
N = math.ceil((M - 1)/2)
c0 = a[0,:]
cn = np.vstack([np.flipud(a[1:N+1,:]),
               np.zeros((1, m+1))])

Pick points not in samples and compare interpolate with actual.

In [ ]:
circ = np.exp(i2pi*np.arange(M, dtype=np.double)/M + 1j/M/2.)
zt = np.empty((M, m+1), dtype=np.complex_)
zt[:,0] = circ
for d, q, j in zip(dv, qv, range(m)):
    zt[:,j+1] = d + q*circ

Interpolate each by column.

In [ ]:
gt = np.empty(zt.shape, dtype=np.complex_)
gt.shape
for j, d, q in zip(range(m+1), np.hstack([0., dv]), np.hstack([1., qv])):
    gt[:,j] = c0[j].real + 2.*polyval(cn[:,j], (zt[:,j] - d)/q).real

In [ ]:
np.max(abs(g(zt) - gt), axis=0)